In [1]:
import numpy as np
from jami import nc, input_to_grid
from pyproj import Proj
from sklearn.neighbors.ball_tree import BinaryTree

In [2]:
# the input data "winGlink_3dmod_input.xyzv" had no information about the crs.
# so, we had to hard code it.
# epsg 32753 represents the data area are in WGS84/ UTM 53S

source_proj = Proj(init='epsg:32753')
grid_proj = Proj(init='epsg:4326')

# make a grid that covers the whole area of all the points
# with desired resolution. The grid epsg we want is 4326 which is global and in lat/lon.
grid_spec = nc.grid_from_extent_and_resolution(left=(131.5, -31., -4500.),
                                               right=(132.5, -30., -125.),
                                               resolution=(0.005, 0.005, 100.))

In [3]:

WING_LINK_DATA_FILE = '../input_files/winGlink_3dmod_input.xyzv'

# input data hes been fixed for false origin
source_points, source_values = input_to_grid.read_winglink_xyzv(WING_LINK_DATA_FILE,
                                                                     false_easting=500000.0,
                                                                     false_northing=10000000.0)

In [4]:
output_file = '../netCDF_files/winGlink.nc'

input_to_grid.mask_interpolate_and_write(output_file,
                                              source_points, source_values, source_proj,
                                              grid_spec, grid_proj)

In [4]:
grid_x, grid_y, grid_z, grid_points = grid_spec
# flatten grid because IDW takes a flattened array as input
utm_points = nc.transform_3d(grid_proj, source_proj, nc.flatten_grid(grid_points))

In [5]:
# mask source for grid
# this is needed because the high resistivity of the air layers contaminate
# the IDW interpolated data underground
source_points_in_grid_proj = nc.transform_3d(source_proj, grid_proj, source_points)
mask = nc.clipping_mask(source_points_in_grid_proj, grid_points)
source_points = source_points[mask]
source_values = source_values[mask]


In [8]:
tree = BinaryTree(source_points, leaf_size=2)

In [9]:
tree.query_radius(query_radius)

In [6]:
# interpolate data at grid points and reshape back to the grid
#resistivity = nc.IDW_NN(source_points, source_values, utm_points)
resistivity = nc.IDW_radius(source_points, source_values, utm_points, radius=1000., p=2)
resistivity = resistivity.reshape(grid_points.shape[:3])


In [8]:
# write to output NetCDF file"
output_file = "../netCDF_files/winGlink_IDW_radius.nc"
nc.write_resistivity_grid(output_file, grid_proj, grid_y, grid_x, grid_z, resistivity.transpose([2, 0, 1]))

